In [1]:
'''
Overview
this notebook is a working example of scraping ufc fight stats one at a time
the code is broken down into small steps that can be run in parts to view and verify results at each step
this is useful for testing and debugging the code as the ufc stats website may be update, breaking the code


scrape ufc fight stats
get all event details, name, url, date, location for all ufc events
for each event, get fight details all fights on card
parse each fight to get fight stats of both fighters

additional scraping of fighter's details, name, nickname, url
for each fighter, get their tale of the tape
'''

'\nOverview\nthis notebook is a working example of scraping ufc fight stats one at a time\nthe code is broken down into small steps that can be run in parts to view and verify results at each step\nthis is useful for testing and debugging the code as the ufc stats website may be update, breaking the code\n\n\nscrape ufc fight stats\nget all event details, name, url, date, location for all ufc events\nfor each event, get fight details all fights on card\nparse each fight to get fight stats of both fighters\n'

In [1]:
# imports
import pandas as pd
import numpy as np
import os
import re
import requests
from bs4 import BeautifulSoup
import itertools

# import library
import scrape_ufc_stats_library as LIB
import importlib
importlib.reload(LIB)

# import configs
import yaml
config = yaml.safe_load(open('scrape_ufc_stats_config.yaml'))

# Parse Event Details
Includes:
<br>
Event
<br>
URL
<br>
Date
<br>
Location
<br>

In [2]:
# define url to parse
url = 'http://ufcstats.com/statistics/events/completed' # first page
# url = 'http://ufcstats.com/statistics/events/completed?page=all' # all pages

# get soup
soup = LIB.get_soup(url)

In [3]:
# parse event details
event_details_df = LIB.parse_event_details(soup)

# show event details
display(event_details_df)

,EVENT,URL,DATE,LOCATION
0,UFC 296: Edwards vs. Covington,http://ufcstats.com/event-details/a8e8587a06e7...,"December 16, 2023","Las Vegas, Nevada, USA"
1,UFC Fight Night: Song vs. Gutierrez,http://ufcstats.com/event-details/5ef0088c1b19...,"December 09, 2023","Las Vegas, Nevada, USA"
2,UFC Fight Night: Dariush vs. Tsarukyan,http://ufcstats.com/event-details/53e4e21f4c68...,"December 02, 2023","Austin, Texas, USA"
3,UFC Fight Night: Allen vs. Craig,http://ufcstats.com/event-details/3feed7d02711...,"November 18, 2023","Las Vegas, Nevada, USA"
4,UFC 295: Prochazka vs. Pereira,http://ufcstats.com/event-details/5a558ba1ff5e...,"November 11, 2023","New York City, New York, USA"
5,UFC Fight Night: Almeida vs. Lewis,http://ufcstats.com/event-details/7c4ec656d8fc...,"November 04, 2023","Sao Paulo, Sao Paulo, Brazil"
6,UFC 294: Makhachev vs. Volkanovski 2,http://ufcstats.com/event-details/13a0fb8fbdaf...,"October 21, 2023","Abu Dhabi, Abu Dhabi, United Arab Emirates"
7,UFC Fight Night: Yusuff vs. Barboza,http://ufcstats.com/event-details/f3a078277b3b...,"October 14, 2023","Las Vegas, Nevada, USA"
8,UFC Fight Night: Dawson vs. Green,http://ufcstats.com/event-details/c8a49ff2acb6...,"October 07, 2023","Las Vegas, Nevada, USA"
9,UFC Fight Night: Fiziev vs. Gamrot,http://ufcstats.com/event-details/c945adc22c2b...,"September 23, 2023","Las Vegas, Nevada, USA"


# Parse Fight Details
Includes:
<br>
Event
<br>
Bout
<br>
URL

In [148]:
# parse one event for fight details

# define url to parse
url = 'http://ufcstats.com/event-details/509697e08673d2e5'
# get soup
soup = LIB.get_soup(url)

In [149]:
# parse fight links
fight_details_df = LIB.parse_fight_details(soup)

# show fight links
display(fight_details_df)

,EVENT,BOUT,URL
0,UFC Fight Night: Font vs. Aldo,Jose Aldo vs. Rob Font,http://ufcstats.com/fight-details/3109d1151f14...
1,UFC Fight Night: Font vs. Aldo,Rafael Fiziev vs. Brad Riddell,http://ufcstats.com/fight-details/a38648a1c190...
2,UFC Fight Night: Font vs. Aldo,Jamahal Hill vs. Jimmy Crute,http://ufcstats.com/fight-details/b3847772199e...
3,UFC Fight Night: Font vs. Aldo,Clay Guida vs. Leonardo Santos,http://ufcstats.com/fight-details/5d64044b9850...
4,UFC Fight Night: Font vs. Aldo,Chris Curtis vs. Brendan Allen,http://ufcstats.com/fight-details/9e5bbbacf1d9...
5,UFC Fight Night: Font vs. Aldo,Alex Morono vs. Mickey Gall,http://ufcstats.com/fight-details/657b7da9c893...
6,UFC Fight Night: Font vs. Aldo,Dusko Todorovic vs. Maki Pitolo,http://ufcstats.com/fight-details/723a8a8c82ca...
7,UFC Fight Night: Font vs. Aldo,Manel Kape vs. Zhalgas Zhumagulov,http://ufcstats.com/fight-details/376dddfa82de...
8,UFC Fight Night: Font vs. Aldo,Bryan Barberena vs. Darian Weeks,http://ufcstats.com/fight-details/405bded9cc7a...
9,UFC Fight Night: Font vs. Aldo,Cheyanne Vlismas vs. Mallory Martin,http://ufcstats.com/fight-details/3fc8afbb23b9...


# Parse Fight Results and Stats

In [150]:
# define url to parse
# various types of fights
# url = 'http://ufcstats.com/fight-details/4b7ec02b39fc6f70' # one round finish
# url = 'http://ufcstats.com/fight-details/8b3b38167060b1d7' # three rounds decision
# url = 'http://ufcstats.com/fight-details/b22eab3aa1522f40' # three rounds finish
# url = 'http://ufcstats.com/fight-details/3109d1151f149aaf' # five rounds decision
# url = 'http://ufcstats.com/fight-details/d93c8c77e1091a16' # no stats
# url = 'http://ufcstats.com/fight-details/c63edd25d2201a46' # draw
url = 'http://ufcstats.com/fight-details/37cb7ce0f0b70640' # point deduction

# get soup
soup = LIB.get_soup(url)

### Parse Fight Results

Includes:
<br>
Event
<br>
Bout
<br>
Weightclass
<br>
Method
<br>
Round
<br>
Time
<br>
Time Format
<br>
Referee
<br>
Details
<br>

In [151]:
# parse fight results from soup
fight_results = LIB.parse_fight_results(soup)
# append fight url
fight_results.append('URL:'+url)

# show fight results
fight_results

['UFC 33: Victory in Vegas ',
 'Dave Menne ',
 'Gil Castillo ',
 'W',
 'L',
 'UFC Middleweight Title Bout',
 'Method: Decision - Unanimous ',
 'Round:5',
 'Time:5:00',
 'Time format:5 Rnd (5-5-5-5-5)',
 'Referee:John McCarthy',
 'Details:Point Deducted: Illegal Knee by MenneTony Weeks 45 - 49.Doug Crosby 42 - 49.Jeff Mullen 44 - 49.',
 'URL:http://ufcstats.com/fight-details/37cb7ce0f0b70640']

In [152]:
# organise fight results
fight_results_df = LIB.organise_fight_results(fight_results, config['fight_results_column_names'])

# show fight results
display(fight_results_df)

,EVENT,BOUT,OUTCOME,WEIGHTCLASS,METHOD,ROUND,TIME,TIME FORMAT,REFEREE,DETAILS,URL
0,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,W/L,UFC Middleweight Title Bout,Decision - Unanimous,5,5:00,5 Rnd (5-5-5-5-5),John McCarthy,Point Deducted: Illegal Knee by MenneTony Week...,http://ufcstats.com/fight-details/37cb7ce0f0b7...


### Parse Fight Stats
Includes:
Event
<br>
Bout
<br>
Round
<br>
Fighter
<br>
Kd
<br>
Sig.Str.
<br>
Sig.Str. %
<br>
Total Str.
<br>
Td
<br>
Td %
<br>
Sub.Att
<br>
Rev.
<br>
Ctrl
<br>
Head
<br>
Body
<br>
Leg
<br>
Distance
<br>
Clinch
<br>
Ground
<br>

In [153]:
# parse full fight stats for both fighters
fighter_a_stats, fighter_b_stats = LIB.parse_fight_stats(soup)

# show fighter stats
print(fighter_a_stats[:20])

['Dave Menne', '0', '70 of 125', '56%', '180 of 241', '1 of 1', '100%', '4', '2', '7:55', 'Dave Menne', '0', '15 of 22', '68%', '45 of 52', '1 of 1', '100%', '1', '0', '2:02']


In [154]:
# organise stats extracted from soup
fighter_a_stats_clean = LIB.organise_fight_stats(fighter_a_stats)
fighter_b_stats_clean = LIB.organise_fight_stats(fighter_b_stats)

# show organised stats
print(fighter_a_stats_clean[:2])

[['Dave Menne', '0', '70 of 125', '56%', '180 of 241', '1 of 1', '100%', '4', '2', '7:55'], ['Dave Menne', '0', '15 of 22', '68%', '45 of 52', '1 of 1', '100%', '1', '0', '2:02']]


In [155]:
# convert list of fighter stats into a structured dataframe
fighter_a_stats_df = LIB.convert_fight_stats_to_df(fighter_a_stats_clean, config['totals_column_names'], config['significant_strikes_column_names'])
fighter_b_stats_df = LIB.convert_fight_stats_to_df(fighter_b_stats_clean, config['totals_column_names'], config['significant_strikes_column_names'])

# show stats df
display(fighter_a_stats_df)

,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
0,Round 1,Dave Menne,0,15 of 22,68%,45 of 52,1 of 1,100%,1,0,2:02,5 of 9,8 of 10,2 of 3,2 of 4,8 of 10,5 of 8
1,Round 2,Dave Menne,0,6 of 15,40%,34 of 45,0 of 0,---,2,1,3:07,3 of 12,2 of 2,1 of 1,2 of 5,1 of 2,3 of 8
2,Round 3,Dave Menne,0,14 of 29,48%,29 of 47,0 of 0,---,0,1,0:14,7 of 21,3 of 3,4 of 5,10 of 24,4 of 5,0 of 0
3,Round 4,Dave Menne,0,21 of 41,51%,24 of 44,0 of 0,---,1,0,0:07,13 of 31,3 of 3,5 of 7,7 of 21,13 of 19,1 of 1
4,Round 5,Dave Menne,0,14 of 18,77%,48 of 53,0 of 0,---,0,0,2:25,10 of 13,0 of 0,4 of 5,3 of 5,4 of 6,7 of 7


In [156]:
# combine fighter stats into one
fight_stats = LIB.combine_fighter_stats_dfs(fighter_a_stats_df, fighter_b_stats_df, soup)

# show fight stats
display(fight_stats)

,EVENT,BOUT,ROUND,FIGHTER,KD,SIG.STR.,SIG.STR. %,TOTAL STR.,TD,TD %,SUB.ATT,REV.,CTRL,HEAD,BODY,LEG,DISTANCE,CLINCH,GROUND
0,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 1,Dave Menne,0,15 of 22,68%,45 of 52,1 of 1,100%,1,0,2:02,5 of 9,8 of 10,2 of 3,2 of 4,8 of 10,5 of 8
1,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 2,Dave Menne,0,6 of 15,40%,34 of 45,0 of 0,---,2,1,3:07,3 of 12,2 of 2,1 of 1,2 of 5,1 of 2,3 of 8
2,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 3,Dave Menne,0,14 of 29,48%,29 of 47,0 of 0,---,0,1,0:14,7 of 21,3 of 3,4 of 5,10 of 24,4 of 5,0 of 0
3,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 4,Dave Menne,0,21 of 41,51%,24 of 44,0 of 0,---,1,0,0:07,13 of 31,3 of 3,5 of 7,7 of 21,13 of 19,1 of 1
4,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 5,Dave Menne,0,14 of 18,77%,48 of 53,0 of 0,---,0,0,2:25,10 of 13,0 of 0,4 of 5,3 of 5,4 of 6,7 of 7
0,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 1,Gil Castillo,0,0 of 0,---,22 of 22,1 of 4,25%,0,0,2:27,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0
1,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 2,Gil Castillo,0,0 of 0,---,15 of 15,3 of 5,60%,0,0,1:05,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0,0 of 0
2,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 3,Gil Castillo,0,4 of 11,36%,38 of 45,1 of 3,33%,0,0,2:47,2 of 9,2 of 2,0 of 0,4 of 11,0 of 0,0 of 0
3,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 4,Gil Castillo,0,4 of 12,33%,21 of 29,1 of 2,50%,0,0,3:11,3 of 10,1 of 2,0 of 0,3 of 11,1 of 1,0 of 0
4,UFC 33: Victory in Vegas,Dave Menne vs. Gil Castillo,Round 5,Gil Castillo,0,1 of 2,50%,14 of 15,1 of 5,20%,0,0,2:06,0 of 1,0 of 0,1 of 1,0 of 1,1 of 1,0 of 0


# Parse Fighter Details

In [56]:
# define url to parse
url = 'http://ufcstats.com/statistics/fighters?char=b&page=all' # 'a' last names
url = 'http://ufcstats.com/statistics/fighters?char=b&page=all' # 'b' last names

In [57]:
# get soup
soup = LIB.get_soup(url)

In [58]:
# parse fighter details
fighter_details_df = LIB.parse_fighter_details(soup, config['fighter_details_column_names'])

# show fighter details
display(fighter_details_df)

,FIRST,LAST,NICKNAME,URL
0,Niklas,Backstrom,,http://ufcstats.com/fighter-details/39cc64bf0a...
1,Seth,Baczynski,The Polish Pistola,http://ufcstats.com/fighter-details/fd5b6598a3...
2,Ryan,Bader,Darth,http://ufcstats.com/fighter-details/c0ab242c40...
3,Izabela,Badurek,,http://ufcstats.com/fighter-details/4fcf6e0c4e...
4,Miguel,Baeza,Caramel Thunder,http://ufcstats.com/fighter-details/82d4c95764...
...,...,...,...,...
267,JP,Buys,Young Savage,http://ufcstats.com/fighter-details/0b36224780...
268,Dennis,Buzukja,The Great,http://ufcstats.com/fighter-details/c4d039123e...
269,Charles,Byrd,Kid Dynamite,http://ufcstats.com/fighter-details/76ee3d666c...
270,Steve,Byrnes,The Sergeant,http://ufcstats.com/fighter-details/4361245697...


# Parse Fighter Tale Of The Tape

In [3]:
# define url to parse
url = 'http://ufcstats.com/fighter-details/93fe7332d16c6ad9' # '--' and '' present
url = 'http://ufcstats.com/fighter-details/b361180739bed4b0' # '--' present
url = 'http://ufcstats.com/fighter-details/1897b7b913736a7c' # no first name
url = 'http://ufcstats.com/fighter-details/d0f3959b4a9747e6' # all tott present
url = 'http://ufcstats.com/fighter-details/4461d7e47375a895' # 平良達郎


In [4]:
# get soup
soup = LIB.get_soup(url)

In [12]:
# parse fighter tale of the tape
fighter_tott = LIB.parse_fighter_tott(soup)

# fighter_tott = [item for item in fighter_tott if item != '']
# show tale of the tape
fighter_tott

['Fighter:Tatsuro Taira',
 'Record:15-0-0',
 'Height:5\' 7"',
 'Weight:125 lbs.',
 'Reach:70"',
 'STANCE:Orthodox',
 'DOB:Jan 27, 2000',
 'SLpM:3.51',
 'Str. Acc.:71%',
 'SApM:1.15',
 'Str. Def:47%',
 'TD Avg.:2.12',
 'TD Acc.:41%',
 'TD Def.:71%',
 'Sub. Avg.:2.4']

In [13]:
# create empty df to store fighters' tale of the tape
# all_fighter_tott_df = pd.DataFrame(columns=config['fighter_tott_column_names'])

# organise fighter tale of the tape
fighter_tott_df = LIB.organise_fighter_tott(fighter_tott, config['fighter_tott_column_names'], url)

# show fighter tale of the tape
display(fighter_tott_df)

,FIGHTER,RECORD,HEIGHT,WEIGHT,REACH,STANCE,DOB,SLpM,Str. Acc.,SApM,Str. Def,TD Avg.,TD Acc.,TD Def.,Sub. Avg.,URL
0,Tatsuro Taira,15-0-0,"5' 7""",125 lbs.,"70""",Orthodox,"Jan 27, 2000",3.51,71%,1.15,47%,2.12,41%,71%,2.4,http://ufcstats.com/fighter-details/4461d7e473...


In [17]:
fighter_tott_clean = [re.sub('^(.+?): ?', '', text) for text in fighter_tott]
fighter_tott_clean

['Tatsuro Taira',
 '5\' 7"',
 '125 lbs.',
 '70"',
 'Orthodox',
 'Jan 27, 2000',
 '3.51',
 '71%',
 '1.15',
 '47%']

In [18]:
fighter_tott_clean.append(url)
fighter_tott_clean

['Tatsuro Taira',
 '5\' 7"',
 '125 lbs.',
 '70"',
 'Orthodox',
 'Jan 27, 2000',
 '3.51',
 '71%',
 '1.15',
 '47%',
 'http://ufcstats.com/fighter-details/4461d7e47375a895']

In [11]:
text = "Record: 15-0-0"

# ':' で分割
parts = text.split(':')

# 分割後の2番目の要素（インデックスは1）が目的のデータ
result = parts[1].strip()  # strip() は前後の余分な空白を削除するために使用

print(result)


15-0-0


In [23]:
fighter_tott_df = pd.DataFrame(columns=config['fighter_tott_column_names'])
fighter_tott_df

,FIGHTER,HEIGHT,WEIGHT,REACH,STANCE,DOB,SLpM,Str. Acc.,SApM,Str. Def,URL
